# Generate files for training and prediction.
# input is WSGA selected columns
# remeber to change prefix to HS or HIS

In [83]:
import csv
import pandas as pd
import pysam
import numpy as np
import sys
import Bio.SubsMat.MatrixInfo

In [85]:
prefix = '.HIS.'
#prefix = '.All.'
#prefix = '.HS.'

In [86]:
FASTA_LOC = '/data/hq2130/large_files/resources/hg19.fasta' # need to modify
REVEL = '/data/hq2130/large_files/revel_file/revel_all_chr.txt.gz'  # REVEL loc
f_revel= pysam.TabixFile(REVEL)
MPC = '/data/hq2130/large_files/fordist_constraint_official_mpc_values.txt.gz'  # mpc
f_MPC = pysam.TabixFile(MPC)

In [115]:

def matrix_score(a_0, a_1, name_matrix="blosum62"):
    """
    Input: a str a_0, a str a_1, a str name_matrix
    Output: The matrix score of a_0 and a_1 in the matrix name_matrix
    """
    # Biopython also included placeholder amino acids (B. J, X, Z) in its
    # scoring matrix.

    matrix = getattr(Bio.SubsMat.MatrixInfo, name_matrix)

    # Since PAM250 in Biopython is not symmetric, if (a_0, a_1) does not exist,
    # matrix_score() will check if (a_1, a_0) exists.

    if (a_0, a_1) in matrix:
        return matrix[(a_0, a_1)]
    elif (a_1, a_0) in matrix:
        return matrix[(a_1, a_0)]
    else:
        return -1
        #raise KeyError("({}, {}) does not exist in matrix.".format(a_0, a_1))

# SCORE = []
# aasets = ['G','A','V','L','I','P','S','D','E','N','Q','K','R','H','F','Y','W','M','C','B','Z','X']
# for a0 in aasets:
#     for a1 in aasets:
#         print a0, a1
#         print matrix_score(a0,a1, 'blosum62')
#         SCORE.append(matrix_score(a0,a1, 'blosum62'))
# print np.median(SCORE) #-1
# print np.mean(SCORE) #~-1

        
        
## correct GCcontent= (pos~pos+10)/5 to (pos-5~pos+5)/10 by cc on 06/29/2017 
def add_gc_content(info):
    chrom, pos = info['hg19_chr'], float(info['hg19_pos(1-based)'])
    fasta_file = FASTA_LOC
    #fasta_file = '/home/local/ARCS/hq2130/Exome_Seq/resources/hg19.fasta' # on server
    fastafile = pysam.Fastafile(fasta_file)
    seq = fastafile.fetch(chrom, pos - 5, pos + 5).upper()
    gc_count = 0
    for dna in seq:
        if dna in {'G', 'C'}:
            #print dna
            gc_count += 1
    gc_count = gc_count / 10.0
    info['gc_content'] = gc_count
    return info

def add_s_het(info):
    gene = info['genename']
    # s_het
    info['s_het'] = 0.01876
    if gene in s_het:
        info['s_het'] = s_het[gene]
    # s_het log, default for log transform of 0
    info['s_het_log'] = np.log(0.01876) 
    if gene in s_het:
        info['s_het_log'] = np.log(s_het[gene]) # minimal value of s_het = 0.000206342
    # info['s_hat_log'] = -10 # default for log transform of 0
    # if gene in s_hat:
    #     info['s_hat_log'] = np.log(s_hat[gene])
    return info


# SCORE = []
# genesets = []
# NA_genenum =0
# with open ('../data/gene/genename_list.txt','rb') as fin:
#     r = csv.reader(fin)
#     for line in r: # line is a list! not a string
#         #print line[0]
#         genesets.append(line)
# for a in genesets:
#     if a[0] in s_het.keys(): # a is a single-item list, not hashable, cannot used as dict keys
#         SCORE.append(s_het[a[0]])
#     else:
#         NA_genenum += 1
# print np.median(SCORE) #0.0187637535
# print np.mean(SCORE) #0.0590276137112
# print np.min(SCORE) #0.000206342
# print NA_genenum #2121

def add_exac_metric(info):
    info['pli'] = pli.get(info['genename'], '0.0277110067492')
    info['lofz'] = lofz.get(info['genename'], '1.98235565023')
    info['prec'] = prec.get(info['genename'], '0.518836940302')
    return info

# SCORE = []
# genesets = []
# NA_genenum =0
# with open ('../data/gene/genename_list.txt','rb') as fin:
#     r = csv.reader(fin)
#     for line in r: # line is a list! not a string
#         #print line[0]
#         genesets.append(line)
# for a in genesets:
#     if a[0] in pli.keys(): # a is a single-item list, not hashable, cannot used as dict keys
#         SCORE.append(pli[a[0]])
#     else:
#         NA_genenum += 1
# print "pli"
# print np.median(SCORE) #0.0277110067492
# print np.mean(SCORE) #0.302933467654
# print np.min(SCORE) #5.35738960908e-91

# SCORE = []
# genesets = []
# NA_genenum =0
# with open ('../data/gene/genename_list.txt','rb') as fin:
#     r = csv.reader(fin)
#     for line in r: # line is a list! not a string
#         #print line[0]
#         genesets.append(line)
# for a in genesets:
#     if a[0] in lofz.keys(): # a is a single-item list, not hashable, cannot used as dict keys
#         SCORE.append(lofz[a[0]])
#     else:
#         NA_genenum += 1
# print "lofz"
# print np.median(SCORE) #1.98235565023
# print np.mean(SCORE) #2.19363750372
# print np.min(SCORE) #-9.7770738652

# SCORE = []
# genesets = []
# NA_genenum =0
# with open ('../data/gene/genename_list.txt','rb') as fin:
#     r = csv.reader(fin)
#     for line in r: # line is a list! not a string
#         #print line[0]
#         genesets.append(line)
# for a in genesets:
#     if a[0] in prec.keys(): # a is a single-item list, not hashable, cannot used as dict keys
#         SCORE.append(prec[a[0]])
#     else:
#         NA_genenum += 1
# print "prec"
# print np.median(SCORE) #0.518836940302
# print np.mean(SCORE) #0.49179236685
# print np.min(SCORE) #6.80216500136e-31

def add_target(info, target):
    info['target'] = target_value
    if target_value == 'NA' and 'cancer_target' in info:
        info['target'] = info['cancer_target']  
    elif target_value == 'NA' and 'category' in info:
        if info['category'] == 'TP':
            info['target'] = 1
        elif info['category'] == 'TN':
            info['target'] = 0 
    return info

def add_gnomad(info):
    info['gnomad'] = 0
    chrom, pos = info['hg19_chr'], info['hg19_pos(1-based)']
    ref, alt = info['ref'], info['alt']
    var_id = '_'.join([chrom, pos, ref, alt])
    if var_id in gnomad_af:
        info['gnomad']= gnomad_af[var_id]    
    return info

def add_secondary(info):
    gene, aaref = info['genename'], info['aaref']
    info['secondary_H'] = 0
    info['secondary_C'] = 0
    info['secondary_E'] = 0
    if gene in secondary:
        aapos = info['aapos'].split(';')
        for pos in aapos:
            pos = int(pos)
            # AA_seq start from 0(it's a list)
            protein_length = len(AA_seq[gene])
            if pos < protein_length and AA_seq[gene][pos-1] == aaref:
                if pos in secondary[gene]:
                    if secondary[gene][pos] == 'H':
                        info['secondary_H'] = 1
                    elif secondary[gene][pos] == 'C':
                        info['secondary_C'] = 1    
                    elif secondary[gene][pos] == 'E':
                        info['secondary_E'] = 1
    return info

SCORE = []
genesets = []
NA_genenum =0
with open ('../data/gene/genename_list.txt','rb') as fin:
    r = csv.reader(fin)
    for line in r: # line is a list! not a string
        #print line[0]
        genesets.append(line)
for a in genesets:
    if a[0] in secondary.keys(): # a is a single-item list, not hashable, cannot used as dict keys
        SCORE.append(secondary[a[0]])
    else:
        NA_genenum += 1
#print "secondary"
#print secondary
#print np.median(SCORE) #0.518836940302
#print np.mean(SCORE) #0.49179236685
#print np.min(SCORE) #6.80216500136e-31

def add_BioPlex(info):
    ''' some feather related to protein? added all pint 
        http://bioplex.hms.harvard.edu/downloadInteractions.php
    '''
    gene = info['genename']
    info['BioPlex'] = 0
    if gene in BioPlex:
        info['BioPlex'] = BioPlex[gene]
    return info

# SCORE = []
# genesets = []
# NA_genenum =0
# with open ('../data/gene/genename_list.txt','rb') as fin:
#     r = csv.reader(fin)
#     for line in r: # line is a list! not a string
#         #print line[0]
#         genesets.append(line)
# for a in genesets:
#     if a[0] not in BioPlex.keys(): # a is a single-item list, not hashable, cannot used as dict keys
# #        print a[0]
# #        SCORE.append(BioPlex[a[0]])
# #    else:
#         NA_genenum += 1
# print "BioPlex"
# #print BioPlex
# #print np.median(SCORE) #5.8844934185
# #print np.mean(SCORE) #9.79657004979
# #print np.min(SCORE) #0.750463933
# print NA_genenum
# print BioPlex['ADA']


def add_REVEL(info, Tabix):
    info['REVEL'] = -1
    chrom, pos = info['hg19_chr'], info['hg19_pos(1-based)']
    ref, alt = info['ref'], info['alt']
    for row in Tabix.fetch(chrom, int(pos)-1, int(pos)+1):# 0-based inputin .fetch
        row = row.split('\t')
        if row[3] == ref and row[4] == alt:
            info['REVEL'] =  row[5]
    return info  

def add_MPC(info, Tabix):
    info['MPC'] = -1
    info['mis_badness'] = -1
    info['obs_exp'] = -1
    chrom, pos = info['hg19_chr'], info['hg19_pos(1-based)']
    ref, alt = info['ref'], info['alt']
    for row in Tabix.fetch(chrom, int(pos)-1, int(pos)+1):# 0-based inputin .fetch
        row = row.split('\t')
        if row[2] == ref and row[3] == alt:
            info['MPC'] = row[-1]
            info['mis_badness'] = row[-3]
            info['obs_exp'] = row[-4]
    return info

# SCORE = []
# genesets = []
# NA_genenum =0
# with open ('../data/gene/genename_list.txt','rb') as fin:
#     r = csv.reader(fin)
#     for line in r: # line is a list! not a string
#         #print line[0]
#         genesets.append(line)
# for a in genesets:
#     if a[0] in prec.keys(): # a is a single-item list, not hashable, cannot used as dict keys
#         SCORE.append(prec[a[0]])
#     else:
#         NA_genenum += 1
# print "prec"
# print np.median(SCORE) #0.518836940302
# print np.mean(SCORE) #0.49179236685
# print np.min(SCORE) #6.80216500136e-31



def choose_HS(info):
    include_variants = False  
    if float(info['pli']) < 0.5 and float(info['pli']) >= 0: # HS genes
        include_variants = True  
    return include_variants

def choose_HIS(info):
    include_variants = False  
    if float(info['pli']) >= 0.5: # HIS genes
        include_variants = True  
    return include_variants

def choose_All(info):
    include_variants = False  
    if float(info['pli']) >= 0: # ALL genes
        include_variants = True  
    return include_variants


BioPlex
7893
1.787753705


In [58]:
def sel_add_feather(fin, w, wgsa_feat, add_feat, extra_feat, target_value, ExAC_AF=0.01, write_head=True):
    """ function that selected colums from wgsa, add some other feathers 
        set . into 0
    """
    with open(fin, 'rU') as f:
        positive, negative = 0, 0
        
        r = csv.reader(f)
        head = r.next()
        feat_all = wgsa_feat + add_feat + extra_feat
        feat = []
        for f in order_feat:
            if f in feat_all:
                feat.append(f)
        for f in feat_all:
            if f not in feat:
                feat.append(f)
        if write_head:
            w.writerow(feat)

        for line in r:
            info = dict(zip(head, line))
            aaref, aaalt, aapos = info['aaref'], info['aaalt'], info['aapos']                
            
            var_id = '_'.join([info['hg19_chr'], info['hg19_pos(1-based)'], info['ref'], info['alt']])
            info['var_id'] = var_id
            if var_id in exclude_var: continue

            # exclude nonsense variants and syn
            if aaref not in {'X', '.'} and aaalt not in [".", 'X']: 
                # reformat wsga feat, missing value filled with 0, will -1 be better?
                for c in wgsa_feat:
                    if info[c] == '.':
                        print c
#                         if c in {'ExAC_AF'}:
#                             info[c] = 0
#                         else:
#                             info[c] = -1
#                     else:
#                         info[c] = float(info[c])
                # set some default value for extra_feat
                for c in extra_feat:
                    info[c] = info.get(c, 'NA')
                
                info['genename'] = info['genename']
                info['blosum62'] = matrix_score(aaref, aaalt, 'blosum62') # function defined in previous cell
                info['pam250'] = matrix_score(aaref, aaalt, 'pam250')
                
                
               
                
                # update SUMO/phospho scores
                info['phospho_score'] = 0
                info['phospho_cutoff'] = 0
                info['phospho_diff'] = 0
                gene = info['genename']
                if gene in phosphorylation:
                    aapos = info['aapos'].split(';')
                    for pos in aapos:
                        pos = int(pos)
                        # pos is 1 based
                        if pos in phosphorylation[gene]:
                            if phosphorylation[gene][pos]['AA'] == aaref:
                                info['phospho_score'] = phosphorylation[gene][pos]['Score']
                                info['phospho_cutoff'] = phosphorylation[gene][pos]['Cutoff']
                                info['phospho_diff'] = phosphorylation[gene][pos]['diff']
                                break
                                
                info['SUMO_score'] = 0
                info['SUMO_cutoff'] = 0
                info['SUMO_diff'] = 0
                if gene in SUMO:
                    aapos = info['aapos'].split(';')
                    for pos in aapos:
                        pos = int(pos)
                        # pos is 1 based
                        if pos in SUMO[gene]:
                            if SUMO[gene][pos]['AA'] == aaref:
                                info['SUMO_score'] = SUMO[gene][pos]['Score']
                                info['SUMO_cutoff'] = SUMO[gene][pos]['Cutoff']
                                info['SUMO_diff'] = SUMO[gene][pos]['diff']
                                break  
                                
                # add inteface flag
                info['interface'] = 0
                if gene in interface:
                    aapos = info['aapos'].split(';')
                    for pos in aapos:
                        pos = int(pos)
                        # AA_seq start from 0
                        protein_length = len(AA_seq[gene])
                        if pos < protein_length and AA_seq[gene][pos-1] == aaref:
                            if pos in interface[gene]:
                                info['interface'] = 1
                                
                # add ASA score Accessible Surface Areas
                info['ASA'] = 0
                if gene in ASA:
                    aapos = info['aapos'].split(';')
                    for pos in aapos:
                        pos = int(pos)
                        # AA_seq start from 0
                        protein_length = len(AA_seq[gene])
                        if pos < protein_length and AA_seq[gene][pos-1] == aaref:
                            if pos in ASA[gene]:
                                info['ASA'] = ASA[gene][pos]
                

                                    
                info['ubiquitination'] = 0
                if gene in ubiquitination:
                    aapos = info['aapos'].split(';')
                    for pos in aapos:
                        pos = int(pos)
                        # AA_seq start from 0
                        protein_length = len(AA_seq[gene])
                        if pos < protein_length and AA_seq[gene][pos-1] == aaref:
                            if pos in ubiquitination[gene]:
                                info['ubiquitination'] = ubiquitination[gene][pos]
                
                # gene specific feathers
                info['complex_CORUM'] = 0
                if gene in complex_CORUM:
                    info['complex_CORUM'] = 1
                
                info['preppi_counts'] = 0
                if gene in preppi:
                    info['preppi_counts'] = preppi[gene]
                    
                info = add_secondary(info)
                info = add_exac_metric(info)
                info = add_gc_content(info)
                info = add_s_het(info)
                info = add_BioPlex(info)
                info = add_MPC(info, f_MPC)
                info = add_REVEL(info, f_revel)
                
                info = add_target(info, target_value)
                info = add_gnomad(info)
               
                # choose variants in HIS or HS
                if prefix == '.HS.':
                    include_variants = choose_HS(info)
                elif prefix == '.HIS.':
                    include_variants = choose_HIS(info)
                elif prefix == '.All.':
                    include_variants = choose_All(info)
                
                # 201707016 remove variants with 0.1% in training/testing
                if float(info['ExAC_AF']) > ExAC_AF:
                    include_variants = False
                    
                    
                if include_variants:
                    if info['target'] == 1:
                        positive += 1
                    if info['target'] == 0:
                        negative += 1
                    
                    w.writerow([info[c] for c in feat])
                    
    print '{} pos, {} neg'.format(positive, negative)

In [59]:
# variants excluded for training
exclude_var = set()
with open('../data/excluded_variants_gwas.txt') as f:
    for line in f:
        exclude_var.add(line.strip())  
        
with open('../data/input_data.exclude.txt') as f:
    for line in f:
        exclude_var.add(line.strip())

In [104]:
# Load protein related annotations ## these are dictionaries !!! not array
SUMO = np.load('../data/protein/SUMO.npy').item()
phosphorylation = np.load('../data/protein/phosphorylation.npy').item()
AA_seq = np.load('../data/protein/AA_seq.npy').item()
interface = np.load('../data/protein/interface.npy').item()
ASA = np.load('../data/protein/ASA.npy').item()
preppi = np.load('../data/protein/preppi.npy').item()
secondary = np.load('../data/protein/secondary.npy').item()
ubiquitination = np.load('../data/protein/ubiquitination.npy').item()
BioPlex = np.load('../data/protein/BioPlex.npy').item()

s_het = np.load('../data/gene/s_het.npy').item()
prec = np.load('../data/gene/prec.npy').item()
pli = np.load('../data/gene/pli.npy').item()


print pli['NME8']
print pli
print 'PREC:'
print prec
print "PLI"
print pli

lofz = np.load('../data/gene/lofz.npy').item()

gnomad_af = np.load('../data/training/gnomad_af.npy').item()


complex_CORUM = set()
with open('../data/protein/protein_complex_CORUM.txt') as f:
    for line in f:
        lst = line.strip().split('\t')
        complex_CORUM = complex_CORUM | set(lst)

2.02524932548e-21


In [ ]:
for i in ['BioPlex' ,'REVEL', "MPC",'']

In [61]:
# feature order from correlation cluster
order_feat = [u'MutationAssessor_score_rankscore', u'VEST3_rankscore', u'Polyphen2_HDIV_rankscore',
 u'Polyphen2_HVAR_rankscore', u'SIFT_converted_rankscore', u'PROVEAN_converted_rankscore',
 u'MetaSVM_rankscore',u'MetaLR_rankscore', u'FATHMM_converted_rankscore', u'M-CAP_rankscore',
 u'GenoCanyon_score_rankscore', u'LRT_converted_rankscore', u'Eigen-PC-raw_rankscore',
 u'Eigen-phred', u'Eigen-PC-phred', u'DANN_rankscore', u'CADD_phred', u'CADD_raw_rankscore',
 u'phyloP20way_mammalian_rankscore', u'GERP++_RS_rankscore', u'SiPhy_29way_logOdds_rankscore',
 u'phastCons100way_vertebrate_rankscore', u'fathmm-MKL_coding_rankscore', u'phyloP100way_vertebrate_rankscore',
 u'MutationTaster_converted_rankscore', u'phastCons20way_mammalian_rankscore', u'GM12878_fitCons_score_rankscore',
 u'HUVEC_fitCons_score_rankscore', u'integrated_fitCons_score_rankscore',u'H1-hESC_fitCons_score_rankscore', 
 u'blosum62', u'pam250', u'SUMO_diff', u'SUMO_score', u'SUMO_cutoff', u'phospho_cutoff', u'phospho_score',
 u'phospho_diff', u'lofz', u'prec', u'pli',
 u's_het', u's_het_log', u'secondary_E', u'secondary_H', u'complex_CORUM', u'preppi_counts',
 u'1000Gp3_AF', u'ExAC_AF', 'gnomad', u'ASA', u'secondary_C', u'gc_content', u'interface', u'ubiquitination']

In [62]:
# add feathers from WGSA and other inputs, some of them need to be excluded in future training
rank_score_cols = ['SIFT_converted_rankscore', 'Polyphen2_HDIV_rankscore', 'Polyphen2_HVAR_rankscore', 
 'LRT_converted_rankscore', 'MutationTaster_converted_rankscore', 'MutationAssessor_score_rankscore', 
 'FATHMM_converted_rankscore', 'PROVEAN_converted_rankscore', 'VEST3_rankscore', 
 'MetaSVM_rankscore', 'MetaLR_rankscore', 'M-CAP_rankscore', 
 'CADD_raw_rankscore', 'DANN_rankscore', 'fathmm-MKL_coding_rankscore', 
 'Eigen-PC-raw_rankscore', 'GenoCanyon_score_rankscore', 'integrated_fitCons_score_rankscore', 
 'GM12878_fitCons_score_rankscore', 'H1-hESC_fitCons_score_rankscore', 
 'HUVEC_fitCons_score_rankscore', 'GERP++_RS_rankscore', 
 'phyloP100way_vertebrate_rankscore', 'phyloP20way_mammalian_rankscore', 
 'phastCons100way_vertebrate_rankscore', 'phastCons20way_mammalian_rankscore', 
 'SiPhy_29way_logOdds_rankscore']

wgsa_feat = ['1000Gp3_AF', 'ExAC_AF', 'CADD_phred', 'Eigen-phred', 'Eigen-PC-phred', 'RVIS']
wgsa_feat = wgsa_feat + rank_score_cols

add_feat =  ['blosum62', 'pam250', 'SUMO_score', 'SUMO_cutoff', 'SUMO_diff',
             'phospho_score', 'phospho_cutoff','phospho_diff', 'interface',
             'ASA', 'pli', 'lofz', 'complex_CORUM', 'preppi_counts',
             'secondary_H', 'secondary_C', 'secondary_E', 'ubiquitination',
             's_het', 'prec', 's_het_log',   'gc_content', 'gnomad', 'BioPlex',
             'obs_exp', 'mis_badness', 'MPC', 'REVEL', 
             'target'] 
              #MPC and REVEL will be excluded cuz models.py self.excluded.features label them with x infront

# feathers used for future info
extra_feat = ['hg19_chr', 'hg19_pos(1-based)', 
              'ref', 'alt', 'category', 'source','INFO', 'disease', 'genename', 'var_id']

for i in {wgsa_feat, add_feat}:
    

# add annotation to testing sets

In [82]:
# # metaSVM and other testing
# fins = ['../data/metaSVM/metaSVM_train.anno.rare.csv', '../data/metaSVM/metaSVM_test1.anno.rare.csv', 
#        '../data/metaSVM/metaSVM_test2.anno.rare.csv', '../data/metaSVM/metaSVM_test3.anno.rare.csv', 
#        '../data/metaSVM/metaSVM_addtest1.anno.rare.csv', '../data/metaSVM/metaSVM_addtest2.anno.rare.csv',
#        '../data/cancer_hotspots/cancer_sel.csv',
#        '../data/gene_test/MCAP_test.anno.rare.csv',
#        '../data/paper_test/ClinVar.anno.rare.csv', '../data/paper_test/UniFun.anno.rare.csv']

# fouts = []
# for f in fins:
#     fouts.append(f.split('.csv')[0] + prefix + 'reformat.csv')
# for fin, fout in zip(fins, fouts):
#     with open(fout, 'w') as fw:
#         w = csv.writer(fw)
#         target_value = 'NA'
#         sel_add_feather(fin, w, wgsa_feat, add_feat, extra_feat, target_value, write_head=True)

5654 pos, 2704 neg
63 pos, 17 neg
2002 pos, 2710 neg
0 pos, 2083 neg
8431 pos, 3062 neg
4718 pos, 2513 neg
0 pos, 0 neg
0 pos, 0 neg
3351 pos, 1547 neg
2964 pos, 770 neg


# add annotation to de novo case and control sets

In [65]:
# # de novo control
# prefixs = ['.HIS.', '.HS.', '.All.']
# for prefix in prefixs:
#     fins = ['../data/case_control/control_900.anno.rare.csv',
#             '../data/case_control/control_1911.anno.rare.csv',
#             '../data/case_control/ssc_yale.anno.rare.csv',
#             '../data/case_control/control_MarkDaly.anno.rare.csv']

#     fouts = [f.split('.csv')[0] + prefix + 'reformat.csv' for f in fins]
#     for fin, fout in zip(fins, fouts):
#         with open(fout, 'w') as fw:
#             w = csv.writer(fw)
#             target_value = 0
#             sel_add_feather(fin, w, wgsa_feat, add_feat, extra_feat, target_value, 
#                             ExAC_AF=1.0/10**5,
#                             write_head=True)

0 pos, 177 neg
0 pos, 331 neg
0 pos, 360 neg
0 pos, 364 neg
0 pos, 298 neg
0 pos, 538 neg
0 pos, 569 neg
0 pos, 591 neg
0 pos, 475 neg
0 pos, 869 neg
0 pos, 929 neg
0 pos, 955 neg


In [66]:
# # de novo case
# prefixs = ['.HIS.', '.HS.', '.All.']
# for prefix in prefixs:
#     fins = ['../data/case_control/case.anno.rare.csv', 
#             '../data/case_control/DDD_new_0.2.anno.rare.csv',
#             '../data/case_control/chd_yale.anno.rare.csv',
#             '../data/case_control/case_MarkDaly.anno.rare.csv',
#             '../data/case_control/CDH_mis.rare.csv']

#     fouts = [f.split('.csv')[0] + prefix + 'reformat.csv' for f in fins]
    
#     for fin, fout in zip(fins, fouts):
#         with open(fout, 'w') as fw:
#             w = csv.writer(fw)
#             target_value = 1
#             sel_add_feather(fin, w, wgsa_feat, add_feat, extra_feat, target_value, 
#                             ExAC_AF=1.0/10**5,
#                             write_head=True)

2163 pos, 0 neg
1557 pos, 0 neg
630 pos, 0 neg
2083 pos, 0 neg
100 pos, 0 neg
2615 pos, 0 neg
1366 pos, 0 neg
847 pos, 0 neg
1863 pos, 0 neg
126 pos, 0 neg
4778 pos, 0 neg
2923 pos, 0 neg
1477 pos, 0 neg
3946 pos, 0 neg
226 pos, 0 neg


# process all missense and all cancer

In [67]:
# # all cancer hostspot and other files
# fins = ['/data/hq2130/large_files/cancer_all.csv']

# fouts = []
# for f in fins:
#     fouts.append(f.split('.csv')[0] + prefix + 'reformat.GCcorrected.csv')
# for fin, fout in zip(fins, fouts):
#     with open(fout, 'w') as fw:
#         w = csv.writer(fw)
#         target_value = 'NA'
#         sel_add_feather(fin, w, wgsa_feat, add_feat, extra_feat, target_value, write_head=True)

In [68]:
# # all cancer hostspot and other files

# prefix = '.All.'
# fins = ['/data/hq2130/large_files/rare_missense_id.anno.rare.csv']

# fouts = []
# for f in fins:
#     fouts.append(f.split('.csv')[0] + prefix + 'reformat.csv')
# for fin, fout in zip(fins, fouts):
#     with open(fout, 'w') as fw:
#         w = csv.writer(fw)
#         target_value = 'NA'
#         sel_add_feather(fin, w, wgsa_feat, add_feat, extra_feat, target_value, 
#                         ExAC_AF=1.0/10**2,
#                         write_head=True)

# print "done" 

# add annotation for different training sets

In [73]:
# prefix = '.HIS.'
# prefix = '.HS.'


# fin = '../data/training/HGMD_DM_missense_norecceive.rare.csv'
# outname = fin.split('.csv')[0] + prefix + 'csv'
# print outname

# with open(outname, 'w') as fw:
#     w = csv.writer(fw)

#     # HGMD positive training
#     #fin = '../data/training/HGMD_DM_missense_anno.rare.csv' 
#     #HGMD positive training ## positive
     
#     target_value = 1
#     sel_add_feather(fin, w, wgsa_feat, add_feat, extra_feat, 
#                     target_value, ExAC_AF=0.01,
#                     write_head=True)
# fw.close()

# ###
# fin = '../data/training/metaSVM_train.anno.rare.csv' 
# outname = fin.split('.csv')[0] + prefix + 'csv'
# print outname

# with open(outname, 'w') as fw:
#     w = csv.writer(fw)
    
#     # metaSVM training ## uniprot positive
    
#     target_value = 'NA'
#     sel_add_feather(fin, w, wgsa_feat, add_feat, extra_feat, target_value, write_head=False)
# fw.close()
    
# ###
# fin = '../data/training/MPC_train.rare.csv' 
# outname = fin.split('.csv')[0] + prefix + 'csv'
# print outname

# with open(outname, 'w') as fw:
#     w = csv.writer(fw)       
#     # MPC train ## from Mark Daly paper 402 HIS positive
#     target_value = 1
#     sel_add_feather(fin, w, wgsa_feat, add_feat, extra_feat, target_value, write_head=True)
# fw.close()

# ###
# fin =  '../data/training/clinvar_pathogenic_1-4star.rare.csv' 
# outname = fin.split('.csv')[0] + prefix + 'csv'
# print outname

# with open(outname, 'w') as fw:
#     w = csv.writer(fw)     
#     # ClinVar from cc as training 
#     target_value = 1
#     sel_add_feather(fin, w, wgsa_feat, add_feat, extra_feat, target_value, write_head=False)
# fw.close()

# ###
# fin = '../data/training/DiscovEHR_rare_missense_30000.csv' 
# outname = fin.split('.csv')[0] + prefix + 'csv'
# print outname

# with open(outname, 'w') as fw:
#     w = csv.writer(fw)     
#     # Discover negative data
#     target_value = 0
#     sel_add_feather(fin, w, wgsa_feat, add_feat, extra_feat, target_value, write_head=False)
# fw.close()


# ###
# fin = '../data/training/CADD_neg_train.anno.rare.csv' 
# outname = fin.split('.csv')[0] + prefix + 'csv'
# print outname

# with open(outname, 'w') as fw:
#     w = csv.writer(fw)     
# #CADD negative data
# # #     
#     target_value = 0
#     sel_add_feather(fin, w, wgsa_feat, add_feat, extra_feat, target_value, write_head=False)
# fw.close()

../data/training/HGMD_DM_missense_norecceive.rare.HS.csv
13633 pos, 0 neg
../data/training/metaSVM_train.anno.rare.HS.csv
7800 pos, 7377 neg
../data/training/MPC_train.rare.HS.csv
14 pos, 0 neg
../data/training/clinvar_pathogenic_1-4star.rare.HS.csv
2347 pos, 0 neg
../data/training/DiscovEHR_rare_missense_30000.HS.csv
0 pos, 18670 neg
../data/training/CADD_neg_train.anno.rare.HS.csv
0 pos, 29729 neg


# decide which training sets to use

In [81]:
prefix = '.HIS.'
#prefix = '.HS.'
fnames = ['../data/training/HGMD_DM_missense_norecceive.rare' + prefix + 'csv',
         '../data/training/metaSVM_train.anno.rare' + prefix + 'csv',
         #'../data/training/MPC_train.rare' + prefix + 'csv' , # 400 3&4 star clinvar
         '../data/training/clinvar_pathogenic_1-4star.rare' + prefix + 'csv' ,
         
          '../data/training/DiscovEHR_rare_missense_30000' + prefix + 'csv' 
          #,'../data/training/CADD_neg_train.anno.rare.csv' 
         ]

outname = '../data/input_data' + prefix + 'csv'
print outname
with open(outname, 'w') as fw:
    w = csv.writer(fw) # create w as an object w for writing 
    flag= 0
    for fname in fnames:
        with open(fname,'rb') as f:
            r = csv.reader(f) # create an object r for read
            if flag == 0:
                head = r.next() 
                print head
                w.writerow(head) # write by rows
                flag = 1
            for line in r: # loop through r by rows
                w.writerow(line)
fw.close()

../data/input_data.HIS.csv
['MutationAssessor_score_rankscore', 'VEST3_rankscore', 'Polyphen2_HDIV_rankscore', 'Polyphen2_HVAR_rankscore', 'SIFT_converted_rankscore', 'PROVEAN_converted_rankscore', 'MetaSVM_rankscore', 'MetaLR_rankscore', 'FATHMM_converted_rankscore', 'M-CAP_rankscore', 'GenoCanyon_score_rankscore', 'LRT_converted_rankscore', 'Eigen-PC-raw_rankscore', 'Eigen-phred', 'Eigen-PC-phred', 'DANN_rankscore', 'CADD_phred', 'CADD_raw_rankscore', 'phyloP20way_mammalian_rankscore', 'GERP++_RS_rankscore', 'SiPhy_29way_logOdds_rankscore', 'phastCons100way_vertebrate_rankscore', 'fathmm-MKL_coding_rankscore', 'phyloP100way_vertebrate_rankscore', 'MutationTaster_converted_rankscore', 'phastCons20way_mammalian_rankscore', 'GM12878_fitCons_score_rankscore', 'HUVEC_fitCons_score_rankscore', 'integrated_fitCons_score_rankscore', 'H1-hESC_fitCons_score_rankscore', 'blosum62', 'pam250', 'SUMO_diff', 'SUMO_score', 'SUMO_cutoff', 'phospho_cutoff', 'phospho_score', 'phospho_diff', 'lofz', 'pr